# Lab. 14 - Raporty i wizualizacja danych

Maciej Wieloch, 303080

AiBD

## Kontrolki interaktywne w Jupyter Notebook
czyli jak korzystać z interaktywnych widgetów IPython, aby usprawnić eksplorację i analizę danych.

W eksploracji danych mało efektywne i zakłócające płynność analizy danych jest wielokrotne uruchamianie tej samej komórki, za każdym razem nieznacznie zmieniając parametry wejściowe, np. wybierając inną wartość funkcji, inne zakresy dat do analizy, czy motyw wizualizacji plotly.

Jednym z rozwiązań tego problemu są interaktywne kontrolki umożliwiające zmianę danych wejściowych bez konieczności przepisywania lub ponownego uruchamiania kodu. Mowa o **IPython widgets** (z biblioteki ipywidgets), które można zbudować za pomocą jednej linii kodu. Biblioteka pozwala nam przekształcić statyczne dokumenty Jupyter Notebook w interaktywne pulpity, idealne do eksploracji i wizualizacji danych.

1. Instalacja: 
```cmd
pip install ipywidgets 
pip install pyarrow
pip install fastparquet
pip install chart_studio
```

1. Aktywacja widgetów dla Jupyter Notebook: 
```cmd
jupyter nbextension enable --py widgetsnbextension
```
2. Import: 
```python
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
```

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import chart_studio.plotly as py

In [3]:
import pandas as pd
import numpy as np

Pokaż wszystkie dane wyjściowe komórek

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Podstawowe widgety
Przed rozpoczęciem wykonywania głównego zadania, proszę zapoznać się z dokumentacją dostępną [tutaj](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html). Zawiera ona przykłady najprostszych widgetów. Dostępna jest również wersja interaktywna, proszę ją wykonać. 

[Tutaj](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) znajduje się lista widgetów. Niech to będzie punkt odniesienia w trakcie pracy z wykorzystaniem widgetów. \
Cała dokumentacja jest dostępna [tutaj](https://ipywidgets.readthedocs.io/en/stable/user_guide.html).

## Wczytanie danych
Dane pochodzą z [repozytorium Willa Koehrsena](https://github.com/WillKoehrsen/Data-Analysis) i zawierają statystyki dotyczące jego artykułów.

Więcej informacji znajdziesz [tutaj](https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6).

In [5]:
data = pd.read_parquet('https://github.com/WillKoehrsen/Data-Analysis/blob/master/medium/data/medium_data_2019_01_26?raw=true')
data.head()

,claps,days_since_publication,fans,link,num_responses,publication,published_date,read_ratio,read_time,reads,...,type,views,word_count,claps_per_word,editing_days,<tag>Education,<tag>Data Science,<tag>Towards Data Science,<tag>Machine Learning,<tag>Python
129,2,597.301123,2,https://medium.com/p/screw-the-environment-but...,0,None,2017-06-10 14:25:00,42.17,7,70,...,published,166,1859,0.001076,0,0,0,0,0,0
125,18,589.983168,3,https://medium.com/p/the-vanquishing-of-war-pl...,0,None,2017-06-17 22:02:00,30.34,14,54,...,published,178,3891,0.004626,0,0,0,0,0,0
132,51,577.363292,20,https://medium.com/p/capstone-project-mercedes...,0,None,2017-06-30 12:55:00,20.02,42,222,...,published,1109,12025,0.004241,0,0,0,0,1,1
126,0,576.520688,0,https://medium.com/p/home-of-the-scared-5af0fe...,0,None,2017-07-01 09:08:00,35.85,9,19,...,published,53,2533,0.000000,0,0,0,0,0,0
121,0,572.533035,0,https://medium.com/p/the-triumph-of-peace-f485...,0,None,2017-07-05 08:51:00,8.47,14,5,...,published,59,3892,0.000000,1,0,0,0,0,0


Proszę zapoznać się z danymi (.columns, .describe(), .info()).

In [6]:
data.columns

Index(['claps', 'days_since_publication', 'fans', 'link', 'num_responses',
       'publication', 'published_date', 'read_ratio', 'read_time', 'reads',
       'started_date', 'tags', 'text', 'title', 'title_word_count', 'type',
       'views', 'word_count', 'claps_per_word', 'editing_days',
       '<tag>Education', '<tag>Data Science', '<tag>Towards Data Science',
       '<tag>Machine Learning', '<tag>Python'],
      dtype='object')

In [7]:
data.describe()

,claps,days_since_publication,fans,num_responses,read_ratio,read_time,reads,title_word_count,views,word_count,claps_per_word,editing_days,<tag>Education,<tag>Data Science,<tag>Towards Data Science,<tag>Machine Learning,<tag>Python
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,1815.263158,248.407273,352.052632,7.045113,29.074662,12.917293,6336.300752,7.127820,23404.030075,3029.120301,0.957638,20.330827,0.729323,0.609023,0.436090,0.383459,0.315789
std,2449.074661,179.370879,479.060117,9.056108,12.417670,9.510795,9007.284726,3.158475,33995.636496,2393.414456,1.846756,74.111579,0.445989,0.489814,0.497774,0.488067,0.466587
min,0.000000,1.218629,0.000000,0.000000,8.110000,1.000000,1.000000,2.000000,3.000000,163.000000,0.000000,-13.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,121.000000,74.543822,23.000000,0.000000,20.020000,8.000000,363.000000,5.000000,1375.000000,1653.000000,0.052115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,815.000000,245.416130,136.000000,4.000000,27.060000,10.000000,2049.000000,7.000000,7608.000000,2456.000000,0.421525,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,2700.000000,376.080598,528.000000,12.000000,34.910000,14.000000,7815.000000,8.000000,30141.000000,3553.000000,1.099366,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,13600.000000,597.301123,2588.000000,59.000000,74.370000,54.000000,41978.000000,16.000000,173714.000000,15063.000000,17.891817,349.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 129 to 17
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   claps                      133 non-null    int64         
 1   days_since_publication     133 non-null    float64       
 2   fans                       133 non-null    int64         
 3   link                       133 non-null    object        
 4   num_responses              133 non-null    int64         
 5   publication                133 non-null    object        
 6   published_date             133 non-null    datetime64[ns]
 7   read_ratio                 133 non-null    float64       
 8   read_time                  133 non-null    int64         
 9   reads                      133 non-null    int64         
 10  started_date               133 non-null    datetime64[ns]
 11  tags                       133 non-null    object        
 12  text   

Stwórz nowy DataFrame, z kolumnami: title, tags, published_date, publication, reads, views, word_count, claps, fans, read_time.

In [9]:
df = data[['title', 'tags', 'published_date', 'publication', 'reads', 'views', 'word_count', 'claps', 'fans', 'read_time']]

df

,title,tags,published_date,publication,reads,views,word_count,claps,fans,read_time
129,"Screw the Environment, but Consider Your Wallet","[Climate Change, Economics]",2017-06-10 14:25:00,None,70,166,1859,2,2,7
125,"The Vanquishing of War, Plague and Famine","[Climate Change, Humanity, Optimism, History]",2017-06-17 22:02:00,None,54,178,3891,18,3,14
132,Capstone Project: Mercedes-Benz Greener Manufa...,"[Machine Learning, Python, Udacity, Kaggle]",2017-06-30 12:55:00,None,222,1109,12025,51,20,42
126,Home of the Scared,"[Politics, Books, News, Media Criticism]",2017-07-01 09:08:00,None,19,53,2533,0,0,9
121,The Triumph of Peace,"[Books, Psychology, History, Humanism]",2017-07-05 08:51:00,None,5,59,3892,0,0,14
...,...,...,...,...,...,...,...,...,...,...
6,The Reality of Global Nuclear Weapons and How ...,"[The Reality Project, Data Journalism, Data, E...",2019-01-12 13:33:00,Towards Data Science,343,1372,2795,370,50,11
3,The Myth of Us vs Them,"[Data, Education, The Reality Project, Data Jo...",2019-01-16 16:44:00,Towards Data Science,266,1005,2532,315,44,10
5,The Poisson Distribution and Poisson Process E...,"[Statistics, Data Science, Towards Data Scienc...",2019-01-20 16:28:00,Towards Data Science,2320,8806,3371,2000,356,14
1,A Great Public Health Conspiracy?,"[Health, Data, Data Journalism, The Reality Pr...",2019-01-25 08:30:00,Towards Data Science,96,559,3549,53,16,13


## Wyświetlanie danych

Za pomocą *.loc* wyświetl artykuły, które zostały przeczytane więcej niż 1000 razy.

In [10]:
df.loc[df['reads'] > 1000]

,title,tags,published_date,publication,reads,views,word_count,claps,fans,read_time
131,Deep Neural Network Classifier,"[Machine Learning, Neural Networks, TensorFlow...",2017-07-25 17:54:00,None,1717,9723,1778,73,20,14
122,Object Recognition with Google’s Convolutional...,"[Machine Learning, Neural Networks, Python, Ob...",2017-07-27 21:17:00,None,5690,24079,2345,240,56,12
127,Facial Recognition Using Google’s Convolutiona...,"[Machine Learning, Python, Neural Networks, Fa...",2017-08-07 14:24:00,None,3992,28387,6666,704,76,38
107,Controlling your Location in Google Chrome,"[Privacy, Security, Google, Internet]",2017-08-15 14:02:00,None,2734,9589,1214,5,5,6
112,Machine Learning with Python on the Enron Dataset,"[Machine Learning, Python, Udacity, Data Analy...",2017-08-20 17:06:00,None,2049,13367,6800,167,32,31
...,...,...,...,...,...,...,...,...,...,...
24,Docker for Data Science Without the Hassle,"[Docker, Data Science, Open Source, Education,...",2018-12-17 20:04:00,Towards Data Science,2135,6766,1075,730,187,5
12,The Copernican Principle and How to Use Statis...,"[Science, Towards Data Science, Education, Sta...",2018-12-29 11:36:00,Towards Data Science,1517,5070,1898,730,155,8
19,The Next Level of Data Visualization in Python,"[Data Science, Data Visualization, Python, Edu...",2019-01-08 22:09:00,Towards Data Science,24002,71225,1457,7600,1697,8
2,A Non-Technical Reading List for Data Science,"[Data Science, Reading, Education, Towards Dat...",2019-01-10 15:14:00,Towards Data Science,2809,15291,3715,1800,369,15


Teraz za pomocą *.loc* wyświetl artykuły, które zostały przeczytane więcej niż 500 razy.

In [11]:
df.loc[df['reads'] > 500]

,title,tags,published_date,publication,reads,views,word_count,claps,fans,read_time
131,Deep Neural Network Classifier,"[Machine Learning, Neural Networks, TensorFlow...",2017-07-25 17:54:00,None,1717,9723,1778,73,20,14
122,Object Recognition with Google’s Convolutional...,"[Machine Learning, Neural Networks, Python, Ob...",2017-07-27 21:17:00,None,5690,24079,2345,240,56,12
127,Facial Recognition Using Google’s Convolutiona...,"[Machine Learning, Python, Neural Networks, Fa...",2017-08-07 14:24:00,None,3992,28387,6666,704,76,38
124,Exploratory Data Analysis with R,"[Data Science, R Programming, Data Analysis, U...",2017-08-08 12:58:00,None,509,4174,7559,58,13,43
119,Data Analysis with Python,"[Python, Data Analysis, Data Visualization, Ba...",2017-08-12 15:17:00,None,664,5581,5508,87,17,29
...,...,...,...,...,...,...,...,...,...,...
12,The Copernican Principle and How to Use Statis...,"[Science, Towards Data Science, Education, Sta...",2018-12-29 11:36:00,Towards Data Science,1517,5070,1898,730,155,8
8,Data Science with Medium Story Stats in Python,"[Data Science, Towards Data Science, Python, E...",2018-12-31 17:53:00,Towards Data Science,754,3768,2813,582,109,12
19,The Next Level of Data Visualization in Python,"[Data Science, Data Visualization, Python, Edu...",2019-01-08 22:09:00,Towards Data Science,24002,71225,1457,7600,1697,8
2,A Non-Technical Reading List for Data Science,"[Data Science, Reading, Education, Towards Dat...",2019-01-10 15:14:00,Towards Data Science,2809,15291,3715,1800,369,15


To najprostszy przykład pokazujący, że interaktywna zmiana parametrów jest potrzebna, by usprawnić proces analizy danych. \
Można to zrobić na przykład za pomocą specjalnej metody, z dekoratorem *@interact*:

In [12]:
from IPython.display import display, HTML

In [13]:
@interact
def show_articles_more_than(column='claps', x=5000):
    display(HTML(f'<h3>Showing articles with more than {x} {column}<h3>'))
    display(df.loc[df[column] > x])

interactive(children=(Text(value='claps', description='column'), IntSlider(value=5000, description='x', max=15…

Dokumentacja [Interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html). Tłumaczy m. in., w jaki sposób parametry funkcji są mapowane do widgetów.

Zauważ, że dekorator *@interact* automatycznie wywnioskował, że potrzebujemy pola tekstowego dla kolumny i suwaka int dla x. 

Gdy potrzebujemy wymusić pewne ograniczenia interakcji, możemy ustawić dodatkowe opcje tworzonej funkcji, takie jak *dropdown* czy granice dla wielkości numerycznych - format to (start, stop, krok):

In [14]:
@interact
def show_titles_more_than(column=list(df.select_dtypes('number').columns), 
                          x=(1000, 5000, 100)):
    display(HTML(f'<h3>Showing articles with more than {x} {column}<h3>'))
    display(df.loc[df[column] > x])

interactive(children=(Dropdown(description='column', options=('reads', 'views', 'word_count', 'claps', 'fans',…

## Dataframe explorer

Stwórz funkcję z dekoratorem *@interact*, żeby szybko znajdować korelację między dwoma wybranymi kolumnami.

In [15]:
@interact
def columns_correlation(column1=list(df.select_dtypes('number').columns), 
                        column2=list(df.select_dtypes('number').columns)):
  display(HTML(f'<h3>Correlation between {column1} and {column2} is: {df[column1].corr(df[column2])}<h3>'))


interactive(children=(Dropdown(description='column1', options=('reads', 'views', 'word_count', 'claps', 'fans'…

Stwórz funkcję z dekoratorem *@interact*, żeby wywołać funkcję *describe()* dla wybranej kolumny.

In [16]:
@interact
def columns_describe(column=list(df.columns)):
  display(HTML(f'<h3>Description of {column} is: <h3>\n<h5>{df[column].describe()}<h5>'))

interactive(children=(Dropdown(description='column', options=('title', 'tags', 'published_date', 'publication'…

## Widgety dla wykresów
Widgety interaktywne są szczególnie przydatne przy wybieraniu danych do wykresu. W tym wypadku również możemy użyć tego samego dekoratora *@interact*, z funkcjami wizualizującymi nasze dane.

**Uwaga.** Obiekt DataFrame nie ma metody iplot, jeśli nie jest połączony z plotly. Potrzebujemy **cufflinks**, aby połączyć pandas z plotly i dodać metodę iplot.

Dodatkowo, aby uniknąć uwierzytelniania, potrzebujemy trybu offline.

In [17]:
import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

In [18]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [19]:
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:]):
    df.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}')

interactive(children=(Dropdown(description='x', options=('reads', 'views', 'word_count', 'claps', 'fans', 'rea…

Dodaj więcej opcji do funkcji scatter_plot:
1. Parametr theme, korzystając z tych dostępnych w cf.themes.THEMES.keys()
2. Parametr colorscale, korzystając z tych dostępnych w cf.colors.\_scales\_names.keys())

In [20]:
@interact
def scatter_plot_w_theme_colorscale(x=list(df.select_dtypes('number').columns), 
                                    y=list(df.select_dtypes('number').columns)[1:],
                                    theme=list(cf.themes.THEMES.keys()),
                                    colorscale=list(cf.colors._scales_names.keys())):
    df.iplot(kind='scatter', x=x, y=y, mode='markers', xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}',
             theme=theme, colorscale=colorscale)

interactive(children=(Dropdown(description='x', options=('reads', 'views', 'word_count', 'claps', 'fans', 'rea…

Do funkcji scatter_plot dodaj parametr categories, grupujący nasze dane. Przetestuj jego działanie.

In [21]:
df['binned_read_time'] = pd.cut(df['read_time'], bins=range(0, 56, 5))
df['binned_read_time'] = df['binned_read_time'].astype(str)

df['binned_word_count'] = pd.cut(df['word_count'], bins=range(0, 100001, 1000))
df['binned_word_count'] = df['binned_word_count'].astype(str)

categories=['binned_read_time', 'binned_word_count', 'publication']

In [22]:
@interact
def scatter_plot_categories(x=list(df.select_dtypes('number').columns), 
                            y=list(df.select_dtypes('number').columns)[1:],
                            theme=list(cf.themes.THEMES.keys()),
                            colorscale=list(cf.colors._scales_names.keys()),
                            categories=['binned_read_time', 'binned_word_count', 'publication', 'types']):
    df.iplot(kind='scatter', x=x, y=y, mode='markers', xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}',
             theme=theme, colorscale=colorscale)

interactive(children=(Dropdown(description='x', options=('reads', 'views', 'word_count', 'claps', 'fans', 'rea…

Być może zauważyłeś, że aktualizacja wykresu przebiegała powoli. W takim przypadku możesz użyć dekoratora *@interact_manual*, który dostarcza przycisku do aktualizacji. 

Sprawdź działanie widgetu z dekoratorem *@interact_manual*.

In [23]:
from ipywidgets import interact_manual

In [24]:
@interact_manual
def scatter_plot_categories(x=list(df.select_dtypes('number').columns), 
                            y=list(df.select_dtypes('number').columns)[1:],
                            theme=list(cf.themes.THEMES.keys()),
                            colorscale=list(cf.colors._scales_names.keys()),
                            categories=['binned_read_time', 'binned_word_count', 'publication', 'types']):
    df.iplot(kind='scatter', x=x, y=y, mode='markers', xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}',
             theme=theme, colorscale=colorscale)

interactive(children=(Dropdown(description='x', options=('reads', 'views', 'word_count', 'claps', 'fans', 'rea…

## Własne widgety
Aby skorzystać jeszcze więcej z biblioteki ipywidgets, możemy sami tworzyć widgety i używać ich w funkcji interakcji.

Stwórz własny widget. Napisz funkcję stats_for_article_published_between, która pobiera datę początkową i końcową, oraz wyświetla statystyki dla wszystkich artykułów opublikowanych między nimi.

In [25]:
df.set_index('published_date', inplace=True)

In [26]:
def stats_for_article_published_between(start_date, end_date):
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    stats = df.loc[(df.index >= start_date) & (df.index <= end_date)].copy()
    num_articles = len(stats)
    total_words = stats['word_count'].sum()
    total_read_time = stats['read_time'].sum()

    print(f'You published {num_articles} articles between {start_date.date()} and {end_date.date()}.')
    print(f'These articles totalled {total_words:,} words and {total_read_time/60:.2f} hours to read.')

Za pomocą następującego kodu funkcja staje się interaktywna:

In [27]:
_ = interact(stats_for_article_published_between,
             start_date=widgets.DatePicker(value=pd.to_datetime('2018-01-01')),
             end_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

interactive(children=(DatePicker(value=Timestamp('2018-01-01 00:00:00'), description='start_date'), DatePicker…

Napisz funkcję plot_up_to, aby narysować wykres kumulatywnej sumy wartości wybranej kolumny, do wybranego dnia.  

Użyj Dropdown i DatePicker w funkcji *interact*.

In [28]:
def plot_up_to(column, date):
    date = pd.Timestamp(date)
    df2plot = df.loc[df.index <= date].copy()

    df2plot[column].cumsum().iplot(mode='markers+lines', xTitle='published date',
                                   yTitle=column, title=f'Cumulative {column.title()} Until {date.date()}')
    
_ = interact(plot_up_to, column=widgets.Dropdown(options=list(df.select_dtypes('number').columns)), 
             date = widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

interactive(children=(Dropdown(description='column', options=('reads', 'views', 'word_count', 'claps', 'fans',…

## Przeglądanie zdjęć

Stwórz funkcję z dekoratorem *@interact*, żeby przeglądać zdjęcia znajdujące się w wybranym folderze. Folder z 3-5 zdjęciami również umieść na repozytorium.

In [29]:
import os
from IPython.display import Image

In [30]:
images_dir = 'images/'
@interact
def show_image(file=os.listdir(images_dir)):
    display(Image(images_dir+file))

interactive(children=(Dropdown(description='file', options=('autumn.jpg', 'spring.jpg', 'summer.jpg', 'winter.…

## Przeglądanie plików

Stwórz funkcję z dekoratorem *@interact*, żeby przeglądać pliki znajdujące się w wybranych folderach.
Skorzystaj z następujących (przykładowych) opcji komendy *ls*: **ls -a -t -r -l -h**. Więcej informacji znajduje się [tutaj](https://www.rapidtables.com/code/linux/ls.html).

In [31]:
import subprocess

In [44]:
@interact
def show_dir(dir=list(filter(os.path.isdir, os.listdir()))):
    x = os.listdir(dir)
    print(x)

interactive(children=(Dropdown(description='dir', options=('.git', '.idea', '.ipynb_checkpoints', 'cities', 'i…

## Zależne widgety
Jeśli chcemy opcje jednego widgetu uzależnić od wartości innego widgetu, używamy metody *observe*. 

Wykorzystaj metodę *observe*, żeby zmienić funkcję przeglądania zdjęć tak, by móc wybierać zarówno ścieżkę, jak i obraz do wyświetlenia. Drugi folder z 3-5 zdjęciami również umieść na repozytorium.

In [33]:
directory = widgets.Dropdown(options=['images', 'cities'])
images = widgets.Dropdown(options=os.listdir(directory.value))

def update_images(*args):
    images.options = os.listdir(directory.value)

directory.observe(update_images, 'value')

def show_images(fdir, file):
    display(Image(f'{fdir}/{file}'))

_ = interact(show_images, fdir=directory, file=images)

interactive(children=(Dropdown(description='fdir', options=('images', 'cities'), value='images'), Dropdown(des…

Możemy również przypisać zmienną do outputu funkcji *interact*, a następnie ponownie użyć widgetu. Może mieć to jednak niezamierzone skutki!

Teraz zmiana wartości w jednej lokalizacji zmienia ją w obu miejscach! Może to być drobna niedogodność, ale zaletą jest to, że możemy ponownie wykorzystać interaktywny element.

In [34]:
dependent_widget = interact(show_images, fdir=directory, file=images)

interactive(children=(Dropdown(description='fdir', options=('images', 'cities'), value='images'), Dropdown(des…

In [35]:
dependent_widget.widget

interactive(children=(Dropdown(description='fdir', options=('images', 'cities'), value='images'), Dropdown(des…

## Wnioski